In [1]:
import pandas as pd
import numpy as np

In [ ]:
"""

Table Schema

t1_user_active_min.csv
This table contains active minutes data logged after experiment started.
Each row represents the total number of minutes spent on site for each user on a date.
If a user never visited the site for a given date, there wouldn't be data for that uid on that date.
- uid: user ID
- dt: date when corresponding active minutes are registered
- active_mins: number of minutes spent on site for the date

t2_user_variant.csv
This table contains users’ treatment assignment.
Each row represents the assignment information for a unique user.
- uid: user ID
- variant_number: the experiment variant user is in. 0 for control, 1 for treatment
- dt: date when user entered the experiment, should be ‘2019-02-06’ for all users
- signup_date: the date string that user signed up on
  
t3_user_active_min_pre.csv
This table contains active minutes data before the experiment started.
It has a similar format as t1, except the dt range can extend before the experiment start date.
- uid: user ID
- dt: date when corresponding active minutes are registered
- active_mins: number of minutes spent on site for the date

t4_user_attributes.csv
This table contains data about some user attributes.
Each row represents attributes of a unique user.
- uid: user ID
- user_type: segment that a user belongs to, measured by activity level of the user. Can be ‘new_user’, ‘non_reader’, ‘reader’ or ‘contributor’
- gender: user gender. Can be ‘male’, ‘female’ or ‘unknown’

"""

In [2]:
df1 = pd.read_csv('/Users/scott/Downloads/quora_data_challenge/t1_user_active_min.csv')

In [3]:
df1.shape

(1066402, 3)

In [4]:
df1.head()

,uid,dt,active_mins
0,0,2019-02-22,5.0
1,0,2019-03-11,5.0
2,0,2019-03-18,3.0
3,0,2019-03-22,4.0
4,0,2019-04-03,9.0


In [6]:
df1.tail()

,uid,dt,active_mins
1066397,49999,2019-04-14,24.0
1066398,49999,2019-04-26,1.0
1066399,49999,2019-05-31,6.0
1066400,49999,2019-06-02,2.0
1066401,49999,2019-06-24,5.0


In [7]:
df2 = pd.read_csv('/Users/scott/Downloads/quora_data_challenge/t2_user_variant.csv')

In [8]:
df2.shape

(50000, 4)

In [9]:
df2.head()

,uid,variant_number,dt,signup_date
0,0,0,2019-02-06,2018-09-24
1,1,0,2019-02-06,2016-11-07
2,2,0,2019-02-06,2018-09-17
3,3,0,2019-02-06,2018-03-04
4,4,0,2019-02-06,2017-03-09


In [10]:
df2.tail()

,uid,variant_number,dt,signup_date
49995,49995,1,2019-02-06,2018-10-11
49996,49996,1,2019-02-06,2014-12-06
49997,49997,1,2019-02-06,2018-11-15
49998,49998,1,2019-02-06,2016-04-05
49999,49999,1,2019-02-06,2015-12-29


In [11]:
df3 = pd.read_csv('/Users/scott/Downloads/quora_data_challenge/t3_user_active_min_pre.csv')

In [12]:
df3.shape

(1190093, 3)

In [13]:
df3.head()

,uid,dt,active_mins
0,0,2018-09-24,3.0
1,0,2018-11-08,4.0
2,0,2018-11-24,3.0
3,0,2018-11-28,6.0
4,0,2018-12-02,6.0


In [14]:
df3.tail()

,uid,dt,active_mins
1190088,49999,2018-09-15,5.0
1190089,49999,2018-09-26,8.0
1190090,49999,2018-10-20,29.0
1190091,49999,2018-12-14,3.0
1190092,49999,2019-01-28,32.0


In [16]:
df4 = pd.read_csv('/Users/scott/Downloads/quora_data_challenge/t4_user_attributes.csv')

In [17]:
df4.shape

(50000, 3)

In [18]:
df4.head()

,uid,gender,user_type
0,0,male,non_reader
1,1,male,reader
2,2,male,non_reader
3,3,male,non_reader
4,4,male,non_reader


In [19]:
df4.tail()

,uid,gender,user_type
49995,49995,unknown,non_reader
49996,49996,male,non_reader
49997,49997,female,reader
49998,49998,male,non_reader
49999,49999,female,non_reader


In [83]:
from scipy import stats 

In [84]:
## Merge tables
new_df1 = pd.merge(df1, df2)
new_df1.head()

,uid,dt,active_mins,variant_number,signup_date
0,17,2019-02-06,157.0,0,2018-06-10
1,18,2019-02-06,1.0,0,2013-06-19
2,38,2019-02-06,26.0,0,2018-10-24
3,50,2019-02-06,1.0,0,2016-08-29
4,52,2019-02-06,7.0,0,2017-07-12


In [85]:
new_df1.shape

(8675, 5)

In [86]:
non_treatment_mins = new_df1.loc[new_df1['variant_number']==0]['active_mins']
non_treatment_mins.head()

0    157.0
1      1.0
2     26.0
3      1.0
4      7.0
Name: active_mins, dtype: float64

In [87]:
non_treatment_mins.describe()

count    7234.000000
mean       19.893420
std        46.827376
min         1.000000
25%         2.000000
50%         4.000000
75%        15.000000
max       894.000000
Name: active_mins, dtype: float64

In [88]:
# Remove the outliers
iqr = 15 - 2 
# print((2 - 1 * iqr_0),(15 + 1.5 * iqr_0))
non_treatment_mins_non_outlier = non_treatment_mins.loc[(non_treatment_mins < (15 + 1.5 * iqr)) & (non_treatment_mins > (2 - 1 *iqr))]
u, std = non_treatment_mins_non_outlier.mean(), non_treatment_mins_non_outlier.std()
print(u, std) 

6.723800353073343 7.605118454744637


In [89]:
treatment_mins = new_df1.loc[new_df1['variant_number']==1]['active_mins']
treatment_mins.head()

7234     1.0
7235    11.0
7236     8.0
7237     6.0
7238     3.0
Name: active_mins, dtype: float64

In [90]:
treatment_mins.describe()

count    1441.000000
mean       28.474670
std        63.661809
min         1.000000
25%         3.000000
50%         8.000000
75%        22.000000
max       669.000000
Name: active_mins, dtype: float64

In [91]:
# Remove the outliers
iqr1 = 22 - 3 
#print((3 - 1.5 * iqr),(22 + 1.5 * iqr))
treatment_mins_non_outlier = treatment_mins.loc[(treatment_mins < (22+ 1.5 * iqr1)) & (treatment_mins > (3 - 1.5 * iqr1))]
u1, std1 = treatment_mins_non_outlier.mean(), treatment_mins_non_outlier.std()
print(u1, std1) 

10.506805444355484 10.956140421867927


In [92]:
stats.norm.interval(0.95, u1, std1) ## 95% confidence interval

(-10.966835192069128, 31.980446080780098)

In [93]:
treatment_mins_non_outlier.shape

(1249,)

In [94]:
stats.ttest_rel(non_treatment_mins_non_outlier[:1249], treatment_mins_non_outlier)

Ttest_relResult(statistic=-10.472262972751212, pvalue=1.1772868452221205e-24)

In [95]:
# Figure out variant_number = 1
treatment_uid = df2[df2['variant_number'] == 1]['uid'] 
treatment_uid

40000    40000
40001    40001
40002    40002
40003    40003
40004    40004
         ...  
49995    49995
49996    49996
49997    49997
49998    49998
49999    49999
Name: uid, Length: 10000, dtype: int64

In [96]:
mins_after = df1.loc[df1['uid'].isin(treatment_uid)]['active_mins']
mins_after

886957      3.0
886958     18.0
886959      4.0
886960     16.0
886961      5.0
           ... 
1066397    24.0
1066398     1.0
1066399     6.0
1066400     2.0
1066401     5.0
Name: active_mins, Length: 179445, dtype: float64

In [97]:
mins_before = df3.loc[df3['uid'].isin(treatment_uid)]['active_mins']
mins_before

989489      1.0
989490      1.0
989491      5.0
989492      3.0
989493      1.0
           ... 
1190088     5.0
1190089     8.0
1190090    29.0
1190091     3.0
1190092    32.0
Name: active_mins, Length: 200604, dtype: float64

In [98]:
mins_before.describe()

count    200604.000000
mean         16.078957
std         500.199145
min           1.000000
25%           2.000000
50%           4.000000
75%          11.000000
max       99999.000000
Name: active_mins, dtype: float64

In [99]:
# Remove the outliers
iqr2 = 11 - 2 
mins_before_non_outlier = mins_before.loc[(mins_before < (11 + 1.5 * iqr2)) & (mins_before > (2 - 1.5 * iqr2))]

u2, std2 = mins_before_non_outlier.mean(), mins_before_non_outlier.std()
print(u2, std2) 

5.313390997917668 5.221983109099401


In [100]:
stats.norm.interval(0.95, u2, std2)

(-4.921507823793655, 15.548289819628991)

In [101]:
mins_after.describe()

count    179445.000000
mean         40.240408
std        1293.703072
min           1.000000
25%           3.000000
50%           7.000000
75%          19.000000
max       99999.000000
Name: active_mins, dtype: float64

In [102]:
# Remove the outliers
iqr3 = 19 - 3
mins_after_non_outlier = mins_after.loc[(mins_after < (19 + 1.5 * iqr3)) & (mins_after > (3 - 1.5 * iqr3))]

u3, std3 = mins_after_non_outlier.mean(), mins_after_non_outlier.std()
print(u3, std3) 

9.033622831173332 9.277109180707694


In [103]:
stats.norm.interval(0.95, u3, std3)

(-9.149177043659634, 27.2164227060063)

In [104]:
mins_after_non_outlier.shape

(155787,)

In [105]:
stats.ttest_rel(mins_before_non_outlier[:155787],mins_after_non_outlier)

Ttest_relResult(statistic=-137.2209270545106, pvalue=0.0)

In [107]:
## Merge the table with df4
new_df1 = pd.merge(df1, df4)
new_df1

,uid,dt,active_mins,gender,user_type
0,0,2019-02-22,5.0,male,non_reader
1,0,2019-03-11,5.0,male,non_reader
2,0,2019-03-18,3.0,male,non_reader
3,0,2019-03-22,4.0,male,non_reader
4,0,2019-04-03,9.0,male,non_reader
...,...,...,...,...,...
1066397,49999,2019-04-14,24.0,female,non_reader
1066398,49999,2019-04-26,1.0,female,non_reader
1066399,49999,2019-05-31,6.0,female,non_reader
1066400,49999,2019-06-02,2.0,female,non_reader


In [109]:
new_df3 = pd.merge(df3, df4)
new_df3

,uid,dt,active_mins,gender,user_type
0,0,2018-09-24,3.0,male,non_reader
1,0,2018-11-08,4.0,male,non_reader
2,0,2018-11-24,3.0,male,non_reader
3,0,2018-11-28,6.0,male,non_reader
4,0,2018-12-02,6.0,male,non_reader
...,...,...,...,...,...
1190088,49999,2018-09-15,5.0,female,non_reader
1190089,49999,2018-09-26,8.0,female,non_reader
1190090,49999,2018-10-20,29.0,female,non_reader
1190091,49999,2018-12-14,3.0,female,non_reader


In [112]:
## Check the association between Gender
male_mins = new_df1.loc[new_df1['gender']=='male']['active_mins']
male_mins.shape

(664085,)

In [113]:
female_mins = new_df1.loc[new_df1['gender']=='female']['active_mins']
female_mins.shape

(263960,)

In [114]:
## There is no siginificant association between Gender
stats.ttest_rel(male_mins[:263960], female_mins)

Ttest_relResult(statistic=-0.8725215555924346, pvalue=0.38292470768041575)

In [116]:
male_mins_after = new_df1.loc[(new_df1['gender']=='male') & (new_df1['uid'].isin(treatment_uid))]['active_mins']
male_mins_after.shape

(110320,)

In [118]:
male_mins_before = new_df3.loc[(new_df3['gender']=='male') & (new_df3['uid'].isin(treatment_uid))]['active_mins']
male_mins_before.shape

(123817,)

In [119]:
## There is siginificant association between treatment groups of Male
stats.ttest_rel(male_mins_before[:110320], male_mins_after)

Ttest_relResult(statistic=-6.555056315082889, pvalue=5.586479902950626e-11)

In [120]:
female_mins_after = new_df1.loc[(new_df1['gender']=='female') & (new_df1['uid'].isin(treatment_uid))]['active_mins']
female_mins_after.shape

(43759,)

In [121]:
female_mins_before = new_df3.loc[(new_df3['gender']=='female')&(new_df3['uid'].isin(treatment_uid))]['active_mins']
female_mins_before.shape

(48185,)

In [122]:
## There is siginificant association between treatment groups of Female
stats.ttest_rel(female_mins_before[:43759], female_mins_after)

Ttest_relResult(statistic=-4.336550433612149, pvalue=1.450594656195817e-05)

In [125]:
new_df1.user_type.unique() 

array(['non_reader', 'reader', 'new_user', 'contributor'], dtype=object)

In [127]:
non_reader_mins = new_df1.loc[new_df1['user_type']=='non_reader']['active_mins']
non_reader_mins.shape

(576074,)

In [128]:
reader_mins = new_df1.loc[new_df1['user_type']=='reader']['active_mins']
reader_mins.shape

(406461,)

In [129]:
## There is siginificant association between treatment groups of Reader
stats.ttest_rel(non_reader_mins[:406461], reader_mins)

Ttest_relResult(statistic=-22.45690855518372, pvalue=1.2816296384759628e-111)

In [131]:
u4, u5 = non_reader_mins.mean(), reader_mins.mean()
print(u4, u5)
## Reader has almost 5 times more mins than Nonreader

8.560384603366929 41.09470527307663


In [132]:
new_mins = new_df1.loc[new_df1['user_type']=='new_user']['active_mins']
new_mins.shape

(17377,)

In [133]:
contributor_mins = new_df1.loc[new_df1['user_type']=='contributor']['active_mins']
contributor_mins.shape

(66490,)

In [134]:
## There is siginificant association between treatment groups of Users
stats.ttest_rel(contributor_mins[:17377], new_mins)

Ttest_relResult(statistic=6.650785041491005, pvalue=3.0022060042882656e-11)

In [136]:
u6, u7 = new_mins.mean(), contributor_mins.mean()
print(u6, u7)
## Contributer has way more average mins than new user

6.379582206364735 253.0313881786735
